Need to create next docker image...

# Docker:

```bash

docker run --rm -d -v $(pwd):/datos --name jupyterlab_postgraphile -p 8000:8000 -p 8989:8888 palmoreck/postgraphile:3.0.16

```

# Example with django and postgreSQL-postgis

using https://github.com/palmoreck/example-django/blob/main/get_started_with_django_part2.ipynb

In [1]:
%%bash
name_django_proj=mydjangoproj

cd $HOME/
mkdir myproj
cd $HOME/myproj
/home/miuser/.local/bin/django-admin startproject $name_django_proj

mv $name_django_proj/manage.py ./
mv $name_django_proj/$name_django_proj/* $name_django_proj
rm -r $name_django_proj/$name_django_proj/

python3 manage.py migrate #if run it will create db.sqlite3 file


Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

In [2]:
%%bash
cd $HOME/myproj
python3 manage.py startapp satellital_images

In [3]:
%%bash
file=$HOME/myproj/mydjangoproj/settings.py
sed -i "/'django.contrib.staticfiles',/a \ \ \ \ 'satellital_images'," $file

In [4]:
%%bash
file=$HOME/myproj/mydjangoproj/settings.py
sed -i "s/'ENGINE': 'django.db.backends.sqlite3',/'ENGINE': 'django.db.backends.postgresql',/" $file
sed -i "s/'NAME': BASE_DIR \/ 'db.sqlite3',/'NAME': 'postgres',/" $file
sed -i "/'NAME': 'postgres',/a \ \ \ \ \ \ \ \ 'USER': 'postgres'," $file
sed -i "/'USER': 'postgres',/a \ \ \ \ \ \ \ \ 'PASSWORD': 'postgres'," $file
sed -i "/'PASSWORD': 'postgres',/a \ \ \ \ \ \ \ \ 'HOST': '172.17.0.1'," $file
sed -i "/'HOST': '172.17.0.1',/a \ \ \ \ \ \ \ \ 'PORT': '5432'," $file

In [6]:
import os

In [7]:
%%file {os.environ["HOME"]}/myproj/satellital_images/models.py
from django.contrib.gis.db import models

class Collection(models.Model):
    collection_type = models.CharField(unique=True, max_length=5)

class Sensor(models.Model):
    sensor_type = models.CharField(unique=True, max_length=10)
    collection_att = models.ManyToManyField("Collection")

class PathRow(models.Model):
    path_row = models.IntegerField(unique=True)
    
class Image(models.Model):
    download_url = models.CharField(max_length=200, default=None)
    sensor_image_att = models.ForeignKey(Sensor, related_name="sensor_images", on_delete=models.CASCADE)
    path_row_image_att = models.ForeignKey(PathRow, related_name="pathrow_images", on_delete=models.CASCADE)
    collection_image_att = models.ForeignKey(Collection, related_name="collection_images", on_delete=models.CASCADE)
    acquisition_date = models.DateTimeField()
    footprint = models.PolygonField()

Overwriting /home/miuser/myproj/satellital_images/models.py


**include img of DB**

In [8]:
%%bash

cd $HOME/myproj

python3 manage.py makemigrations 

#python3 manage.py makemigrations satellital_images

Traceback (most recent call last):
  File "manage.py", line 22, in <module>
    main()
  File "manage.py", line 18, in main
    execute_from_command_line(sys.argv)
  File "/home/miuser/.local/lib/python3.6/site-packages/django/core/management/__init__.py", line 401, in execute_from_command_line
    utility.execute()
  File "/home/miuser/.local/lib/python3.6/site-packages/django/core/management/__init__.py", line 377, in execute
    django.setup()
  File "/home/miuser/.local/lib/python3.6/site-packages/django/__init__.py", line 24, in setup
    apps.populate(settings.INSTALLED_APPS)
  File "/home/miuser/.local/lib/python3.6/site-packages/django/apps/registry.py", line 114, in populate
    app_config.import_models()
  File "/home/miuser/.local/lib/python3.6/site-packages/django/apps/config.py", line 211, in import_models
    self.models_module = import_module(models_module_name)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_impor

CalledProcessError: Command 'b'\ncd $HOME/myproj\n\npython3 manage.py makemigrations \n\n#python3 manage.py makemigrations satellital_images\n'' returned non-zero exit status 1.

In [7]:
%%bash

cd $HOME/myproj/

python3 manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, satellital_images, sessions
Running migrations:
  Applying satellital_images.0001_initial... OK


**Cmd to see query in sql: python3 manage.py sqlmigrate satellital_images 0001_initial.py**

In [8]:
direc = "/home/miuser"
dir_json_scenes = os.path.join(os.getcwd(), "metadata_scenes_downloaded")
direc_myproj = os.path.join(direc, "myproj")

In [9]:
os.chdir(direc_myproj)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mydjangoproj.settings")
import django
django.setup()
from satellital_images.models import Collection, PathRow, Sensor

In [1]:
import requests

In [2]:
url = "https://raw.githubusercontent.com/palmoreck/example-django/main/metadata_scenes_downloaded/"

scene = "scene0.txt"

In [3]:
scene_url = os.path.join(url, scene)

In [4]:
r = requests.get(scene_url)

In [6]:
scene = r.json()

In [ ]:
scene

In [8]:
download_url = scene["downloadUrl"]
d = scene["displayId"]
list_d = d.split("_")
sensor=list_d[0]
pathrow = list_d[2]
collection=list_d[5]
acq_date = scene["acquisitionDate"]

In [17]:
coll = Collection(collection_type=collection)
pr = PathRow(path_row=pathrow)
sen = Sensor(sensor_type=sensor)

In [18]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
coll.save()
pr.save()
sen.save()
sen.collection_att.add(coll)

In [19]:
from satellital_images.models import Image

In [20]:
img = Image(download_url=download_url,
            sensor_image_att=sen,
            path_row_image_att=pr,
            collection_image_att=coll,
            acquisition_date=acq_date)

In [21]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
img.save()

/home/miuser/.local/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1370: RuntimeWarning: DateTimeField Image.acquisition_date received a naive datetime (2013-03-10 00:00:00) while time zone support is active.
  RuntimeWarning)


In [22]:
import sqlite3

connection = sqlite3.connect("db.sqlite3")
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())

[('django_migrations',), ('sqlite_sequence',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('django_content_type',), ('auth_permission',), ('auth_group',), ('auth_user',), ('django_session',), ('satellital_images_collection',), ('satellital_images_pathrow',), ('satellital_images_sensor',), ('satellital_images_sensor_collection_att',), ('satellital_images_image',)]


**Next just to check, normally is not necessary to execute:**

In [23]:
rows = cursor.execute("SELECT * FROM satellital_images_image").fetchall()
print(rows)

[(1, 'https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80320382013069LGN02/EE/', '2013-03-10 00:00:00', 1, 1, 1)]


In [24]:
rows = cursor.execute("SELECT * FROM satellital_images_collection").fetchall()
print(rows)

[(1, '01')]


In [25]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor").fetchall()
print(rows)

[(1, 'LT08')]


In [26]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor_collection_att").fetchall()
print(rows)

[(1, 1, 1)]


In [27]:
rows = cursor.execute("SELECT * FROM satellital_images_pathrow").fetchall()
print(rows)

[(1, 32038)]


**If we want to change path_row attribute (column) of PathRow class:**

In [28]:
%%file {os.environ["HOME"]}/myproj/satellital_images/models.py
from django.db import models

class Collection(models.Model):
    collection_type = models.CharField(unique=True, max_length=5)

class Sensor(models.Model):
    sensor_type = models.CharField(unique=True, max_length=10)
    collection_att = models.ManyToManyField("Collection")

class PathRow(models.Model):
    path_row = models.CharField(unique=True,max_length=10)#here the change
    
class Image(models.Model):
    download_url = models.CharField(max_length=200, default=None)
    sensor_image_att = models.ForeignKey(Sensor, related_name="sensor_images", on_delete=models.CASCADE)
    path_row_image_att = models.ForeignKey(PathRow, related_name="pathrow_images", on_delete=models.CASCADE)
    collection_image_att = models.ForeignKey(Collection, related_name="collection_images", on_delete=models.CASCADE)
    acquisition_date = models.DateTimeField()
    

Overwriting /home/miuser/example-django/myproj/satellital_images/models.py


**Another scene:**

In [2]:
scene = "scene1.txt"

In [3]:
scene_url = os.path.join(url, scene)

In [4]:
r = requests.get(scene_url)

In [5]:
r

<Response [200]>

In [6]:
scene = r.json()

In [33]:
scene 

{'acquisitionDate': '2013-03-11',
 'startTime': '2013-03-11',
 'endTime': '2013-03-11',
 'lowerLeftCoordinate': {'latitude': 32.50897, 'longitude': -90.64024},
 'upperLeftCoordinate': {'latitude': 34.20572, 'longitude': -90.19275},
 'upperRightCoordinate': {'latitude': 33.83685, 'longitude': -88.27833},
 'lowerRightCoordinate': {'latitude': 32.14512, 'longitude': -88.7621},
 'spatialFootprint': {'type': 'Polygon',
  'coordinates': [[[-90.64024, 32.50897],
    [-88.7621, 32.14512],
    [-88.27833, 33.83685],
    [-90.19275, 34.20572],
    [-90.64024, 32.50897]]]},
 'sceneBounds': '-90.64024,32.14512,-88.27833,34.20572',
 'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_8_c1/2013/022/037/LT08_L1GT_022037_20130311_20170310_01_T2.jpg',
 'dataAccessUrl': 'https://earthexplorer.usgs.gov/metadata/full/landsat_8_c1/LT80220372013070LGN02/',
 'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80220372013070LGN02/EE/',
 'entityId': 'LT80220372013070LGN02',

In [34]:
download_url = scene["downloadUrl"]
d = scene["displayId"]
list_d = d.split("_")
sensor=list_d[0]
pathrow = list_d[2]
collection=list_d[5]
acq_date = scene["acquisitionDate"]

In [35]:
pathrow

'022037'

In [36]:
collection

'01'

In [37]:
sensor

'LT08'

In [38]:
coll = Collection.objects.get_or_create(collection_type=collection)
pr = PathRow.objects.get_or_create(path_row=pathrow)
sen = Sensor.objects.get_or_create(sensor_type=sensor)

In [39]:
coll

(<Collection: Collection object (1)>, False)

In [40]:
pr

(<PathRow: PathRow object (2)>, True)

In [41]:
sen

(<Sensor: Sensor object (1)>, False)

**True was created a new row, False just get the row**

In [42]:
coll = coll[0]
pr = pr[0]
sen = sen[0]

In [43]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
coll.save()
pr.save()
sen.save()
sen.collection_att.add(coll)

In [44]:
img = Image(download_url=download_url,
            sensor_image_att=sen,
            path_row_image_att=pr,
            collection_image_att=coll,
            acquisition_date=acq_date)

In [45]:
#next set default to avoid SynchronousOnlyOperation error:
#You cannot call this from an async context - use a thread or sync_to_async.
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
img.save()

/home/miuser/.local/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1370: RuntimeWarning: DateTimeField Image.acquisition_date received a naive datetime (2013-03-11 00:00:00) while time zone support is active.
  RuntimeWarning)


In [46]:
connection = sqlite3.connect("db.sqlite3")
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())

[('django_migrations',), ('sqlite_sequence',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('django_content_type',), ('auth_permission',), ('auth_group',), ('auth_user',), ('django_session',), ('satellital_images_collection',), ('satellital_images_sensor',), ('satellital_images_sensor_collection_att',), ('satellital_images_image',), ('satellital_images_pathrow',)]


**Next just to check, normally is not necessary to execute:**

In [47]:
rows = cursor.execute("SELECT * FROM satellital_images_image").fetchall()
print(rows)

[(1, 'https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80320382013069LGN02/EE/', '2013-03-10 00:00:00', 1, 1, 1), (2, 'https://earthexplorer.usgs.gov/download/external/options/landsat_8_c1/LT80220372013070LGN02/EE/', '2013-03-11 00:00:00', 1, 2, 1)]


In [48]:
rows = cursor.execute("SELECT * FROM satellital_images_collection").fetchall()
print(rows)

[(1, '01')]


In [49]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor").fetchall()
print(rows)

[(1, 'LT08')]


In [50]:
rows = cursor.execute("SELECT * FROM satellital_images_sensor_collection_att").fetchall()
print(rows)

[(1, 1, 1)]


In [51]:
rows = cursor.execute("SELECT * FROM satellital_images_pathrow").fetchall()
print(rows)

[(1, '32038'), (2, '22037')]
